# **Definición de Endpoints para Análisis de Datos de Steam Games**

Para nuestra API, desarrollaremos funciones especializadas para los siguientes endpoints:

1. *@app.get('/playtime_genre')*
**Función PlayTimeGenre(genero: str):** Retorna el año con más horas jugadas para un género.

2. *@app.get('/user_for_genre')*
**Función UserForGenre(genero: str):** Devuelve el usuario con más horas en un género y su distribución anual de horas jugadas.

3. *@app.get('/users_recommend')*
**Función UsersRecommend(año: int):** Lista el top 3 de juegos más recomendados en un año.

4. *@app.get('/users_worst_developer')*
**Función UsersWorstDeveloper(año: int):** Muestra las 3 desarrolladoras con menos recomendaciones en un año.

5. *@app.get('/sentiment_analysis')*
**Función SentimentAnalysis(empresa: str):** Presenta un análisis de sentimiento de reseñas por desarrolladora.
Cada función aprovechará nuestra base de datos previamente limpiada para generar insights significativos.

In [2]:
#Importamos las librerias
import pandas as pd
import numpy as np 
import pyarrow as pa
import pyarrow.parquet as pq
from fastapi import FastAPI
import unicodedata
app = FastAPI()

In [103]:
pip install uvicorn



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


**Extracción de los datos**

In [3]:
# Carga de datos de juegos de Steam
steam_games = pd.read_csv('data/steam_games_cleaned.csv')

# Carga de datos de reseñas de usuarios
user_reviews = pd.read_csv('data/user_reviews_cleaned.csv')

# Carga de datos de artículos de usuarios
user_items = pd.read_parquet('data/user_items_cleaned.parquet')

### 1. Función **def PlayTimeGenre**(genero:str):

Para analizar las preferencias de juego, utilizaremos la tabla steam_games para el género y año de lanzamiento, y la tabla user_items para las horas jugadas (playtime_forever). Estos datos nos ayudarán a identificar tendencias y patrones de uso.

**Crear una tabla con las columnas que necesitamos para obtener los datos que necesitamos**

In [4]:
#Revisar la tabla steam_games
steam_games.head(3)

,item_id,app_name,title,genres,year_of_release,publisher,developer,specs,price
0,761140.0,Lost Summoner Kitty,Lost Summoner Kitty,Action,2018,Kotoshiro,Kotoshiro,['Single-player'],4.99
1,761140.0,Lost Summoner Kitty,Lost Summoner Kitty,Casual,2018,Kotoshiro,Kotoshiro,['Single-player'],4.99
2,761140.0,Lost Summoner Kitty,Lost Summoner Kitty,Indie,2018,Kotoshiro,Kotoshiro,['Single-player'],4.99


In [5]:
#Extraer las columnas necesarias para esta función
genre= steam_games[["item_id","genres","year_of_release"]]

In [6]:
#Revisar la tabla user_items
user_items.head(3)

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7,0


In [7]:
#Seleccionar las columnas que necesito
play_time= user_items[["item_id","playtime_forever"]]

In [9]:
#Unificar las nuevas tablas creadas
genre_funcion= play_time.merge(genre, on="item_id")
#Revisar la nueva tabla
genre_funcion.head(3)

ValueError: You are trying to merge on object and float64 columns for key 'item_id'. If you wish to proceed you should use pd.concat

**Obtener los datos que necesitamos para nuestra función**

In [ ]:
# Agrupamos los datos por género y año de lanzamiento y sumamos las horas jugadas
total_playtime_by_year_genre = genre_funcion.groupby(["genres", "year_of_release"]).playtime_forever.sum().reset_index(name='total_playtime')

# Ordenamos los resultados por el total de horas jugadas de manera descendente
sorted_playtime = total_playtime_by_year_genre.sort_values(by="total_playtime", ascending=False)

# Muestra los resultados
sorted_playtime

,genres,year_of_release,total_playtime
27,Action,2012,1085635110
163,Indie,2006,446115272
265,Simulation,2006,444536391
28,Action,2013,415522232
26,Action,2011,331697668
...,...,...,...
127,Education,2010,9
247,Simulation,1988,1
279,Software Training,2010,0
303,Strategy,1991,0


**Definir el Máximo de Horas Jugadas por Género de Videojuegos**

In [37]:
# Agrupamos por género y calculamos el máximo de horas jugadas
max_playtime_by_genre = sorted_playtime.groupby("genres")["total_playtime"].agg(MaxHours="max").reset_index()

# Mostramos los primeros registros del resultado
max_playtime_by_genre.head()

,genres,MaxHours
0,Action,1085635110
1,Adventure,221707756
2,Animation &amp; Modeling,1345545
3,Audio Production,455463
4,Casual,81708365


**Encontrar el maximo de horas jugadas por género**

In [38]:
# Agrupamos los datos por género y calculamos el máximo de horas jugadas para cada género
max_hours_per_genre = sorted_playtime.groupby("genres")["total_playtime"].agg(MaxHours="max").reset_index()

# Mostramos los primeros cinco registros para obtener una vista previa del resultado
max_hours_per_genre.head()

,genres,MaxHours
0,Action,1085635110
1,Adventure,221707756
2,Animation &amp; Modeling,1345545
3,Audio Production,455463
4,Casual,81708365


In [39]:
# Obtenemos las dimensiones del DataFrame
max_hours_per_genre.shape

(21, 2)

Renombramos la columna para facilitar el join con max_hours_per_genre. Esto asegura que las columnas tengan nombres consistentes para el merge.

In [40]:
# Renombrar columna para el merge
genre_playtime = sorted_playtime.rename(columns={'total_playtime': 'MaxHours'})

# Mostrar DataFrame actualizado
genre_playtime

,genres,year_of_release,MaxHours
27,Action,2012,1085635110
163,Indie,2006,446115272
265,Simulation,2006,444536391
28,Action,2013,415522232
26,Action,2011,331697668
...,...,...,...
127,Education,2010,9
247,Simulation,1988,1
279,Software Training,2010,0
303,Strategy,1991,0


In [41]:
# Realizamos un merge entre max_hours_per_genre y genre_playtime
merged_genre_data = pd.merge(max_hours_per_genre, genre_playtime)

# Mostramos el DataFrame resultante
merged_genre_data

,genres,MaxHours,year_of_release
0,Action,1085635110,2012
1,Adventure,221707756,2011
2,Animation &amp; Modeling,1345545,2015
3,Audio Production,455463,2014
4,Casual,81708365,2015
5,Design &amp; Illustration,1930339,2012
6,Early Access,118902893,2013
7,Education,340567,2013
8,Free to Play,146213734,2013
9,Indie,446115272,2006


In [42]:
# Guarda el DataFrame 'tabla_final' en un archivo CSV
merged_genre_data_2 = "funcion_1.csv"
merged_genre_data.to_csv(merged_genre_data_2, index=False, encoding="utf-8")

In [43]:
# Convertir un archivo CSV a Parquet

# Lee el archivo CSV
funcion_1 = pd.read_csv("funcion_1.csv")

# Define el nombre y la ubicación del archivo Parquet
genre_file = "data/funcion_1.parquet"

# Convierte el DataFrame a una tabla de PyArrow y guarda en formato Parquet
table = pa.Table.from_pandas(funcion_1)
pq.write_table(table, genre_file)

In [57]:
@app.get('/PlayTimeGenre/{genero}')
def PlayTimeGenre(genero):
    # Convertimos tanto el género de entrada como la columna del DataFrame a minúsculas
    genero_min = genero.lower()
    merged_genre_data["genres"] = merged_genre_data["genres"].str.lower()

    # Filtra los datos para el género especificado
    genre_data = merged_genre_data[merged_genre_data["genres"] == genero_min]
    
    # Encuentra el año con más horas jugadas
    if not genre_data.empty:
        max_playtime_row = genre_data.sort_values(by='MaxHours', ascending=False).iloc[0]

        # Devuelve el año de lanzamiento con más horas jugadas para el género dado
        return {"Año de lanzamiento con más horas jugadas para Género " + genero: int(max_playtime_row["year_of_release"])}
    else:
        return {"Error": "Género no encontrado o sin datos"}

In [45]:
PlayTimeGenre('Indie')

{'Año de lanzamiento con más horas jugadas para Género Indie': 2006}

### 2. Función **def UserForGenre(genero:str)**:

Para crear esta función, continuamos con nuestra tabla que combina información de Steam e Items, incorporando ahora el user_id. Dado que user_id se encuentra en formato de objeto, es necesario convertirlo a cadena de texto (string). Esta conversión implica revisar y posiblemente adaptar la funcionalidad que se desarrolló anteriormente en la Función 1 para asegurar la compatibilidad y precisión de los datos.

In [46]:
# Convertir 'user_id' de object a string
user_items['user_id'] = user_items['user_id'].astype('string')

# Seleccionar columnas relevantes
play_time_user = user_items[["item_id", "playtime_forever", "user_id"]]

# Realizar un merge con el DataFrame 'genre' basado en 'item_id'
combined_genre_user = play_time_user.merge(genre, on="item_id")

# Convertir 'genres', 'Año de lanzamiento' y 'user_id' a string
combined_genre_user['genres'] = combined_genre_user['genres'].astype('string')
combined_genre_user['year_of_release'] = combined_genre_user['year_of_release'].astype('string')
combined_genre_user['user_id'] = combined_genre_user['user_id'].astype('string')

# Visualizar las primeras 5 filas del DataFrame combinado
combined_genre_user.head(5)


,item_id,playtime_forever,user_id,genres,year_of_release
0,10,6,76561197970982479,Action,2000
1,10,0,js41637,Action,2000
2,10,0,Riot-Punch,Action,2000
3,10,93,doctr,Action,2000
4,10,108,corrupted_soul,Action,2000


In [47]:
# Agrupar por género, usuario y año de lanzamiento, y sumar las horas jugadas
total_playtime_by_user_year = combined_genre_user.groupby(["genres", "user_id", "year_of_release"])["playtime_forever"].sum().reset_index()

# Visualizar el DataFrame resultante
total_playtime_by_user_year

,genres,user_id,year_of_release,playtime_forever
0,Action,--000--,2009,5329
1,Action,--000--,2010,22
2,Action,--000--,2011,6522
3,Action,--000--,2012,109346
4,Action,--000--,2013,363
...,...,...,...,...
3449993,Web Publishing,zepavil,2015,9010
3449994,Web Publishing,zeshirky,2007,1
3449995,Web Publishing,zevlupine,2012,4
3449996,Web Publishing,zilaman,2013,9


Ahora vamos a identificar los patrones de juego de los usuarios en relación con diferentes géneros de videojuegos. El objetivo es determinar cuáles usuarios dedican más tiempo a cada género y cómo se distribuye ese tiempo a lo largo de los años.

In [48]:
# Agrupar por género y usuario para sumar las horas jugadas
total_hours_by_user = combined_genre_user.groupby(["genres", "user_id"])["playtime_forever"].sum().reset_index()
total_hours_by_user.head()  # Vista previa de los datos agregados por usuario


,genres,user_id,playtime_forever
0,Action,--000--,139469
1,Action,--ace--,69325
2,Action,--ionex--,38315
3,Action,-2SV-vuLB-Kg,42500
4,Action,-404PageNotFound-,117423


In [49]:
# Encontrar el usuario con la máxima cantidad de horas jugadas por género
max_hours_by_genre = total_hours_by_user.groupby("genres")["playtime_forever"].agg(playtime_forever="max").reset_index()
max_hours_by_genre.head()

,genres,playtime_forever
0,Action,1699307
1,Adventure,2191551
2,Animation &amp; Modeling,168314
3,Audio Production,109916
4,Casual,1224933


In [50]:
# Combinar para obtener el detalle del usuario con más horas jugadas por género
detailed_max_hours = pd.merge(max_hours_by_genre, total_hours_by_user)
detailed_max_hours

,genres,playtime_forever,user_id
0,Action,1699307,Sp3ctre
1,Adventure,2191551,REBAS_AS_F-T
2,Animation &amp; Modeling,168314,ScottyG555
3,Audio Production,109916,Lickidactyl
4,Casual,1224933,REBAS_AS_F-T
5,Design &amp; Illustration,168314,ScottyG555
6,Early Access,316969,76561197978756659
7,Education,65427,76561198059330972
8,Free to Play,808241,idonothack
9,Indie,2402994,REBAS_AS_F-T


In [51]:
# Preparar para agregar el total de horas jugadas por año
playtime_per_year = total_playtime_by_user_year.rename(columns={"playtime_forever": "playtime_forever_year"})
playtime_per_year.head(5)  # Vista previa de los datos con horas jugadas por año

,genres,user_id,year_of_release,playtime_forever_year
0,Action,--000--,2009,5329
1,Action,--000--,2010,22
2,Action,--000--,2011,6522
3,Action,--000--,2012,109346
4,Action,--000--,2013,363


In [52]:
# Verificar el cruce de datos para el usuario específico 'ScottyG555' en el género 'Video Production'
test_user_playtime = playtime_per_year[(playtime_per_year['user_id'] == 'ScottyG555') & (playtime_per_year['genres'] == 'Video Production')]

# Visualizar los resultados de la prueba
test_user_playtime

,genres,user_id,year_of_release,playtime_forever_year
3444538,Video Production,ScottyG555,2015,168314


In [53]:
# Combinar tabla_user con playtime_per_year para mayor detalle
combined_user_data = pd.merge(detailed_max_hours, playtime_per_year)
combined_user_data

,genres,playtime_forever,user_id,year_of_release,playtime_forever_year
0,Action,1699307,Sp3ctre,0,1657
1,Action,1699307,Sp3ctre,1993,0
2,Action,1699307,Sp3ctre,1995,217
3,Action,1699307,Sp3ctre,1996,0
4,Action,1699307,Sp3ctre,1998,0
...,...,...,...,...,...
165,Utilities,207651,76561198073642113,2014,207651
166,Video Production,168314,ScottyG555,2015,168314
167,Web Publishing,142964,Xyphien,2005,7296
168,Web Publishing,142964,Xyphien,2012,64657


In [54]:

# Verificar los registros para 'ScottyG555' en el género 'Video Production'
test_combined_data = combined_user_data[(combined_user_data['user_id'] == 'ScottyG555') & (combined_user_data['genres'] == 'Video Production')]
test_combined_shape = test_combined_data.shape  # Obtener la cantidad de registros

# Resultado de la verificación, tiene que dar el resultado 7
test_combined_shape

(1, 5)

Ajustamos y combinamos datasets para analizar las horas de juego por género y usuario. Verificación de Datos: Confirmamos la precisión de los datos para la creación de funciones.

In [55]:
# Renombrar columnas y preparar el DataFrame
combined_user_playtime = combined_user_data.rename(columns={'year_of_release': 'Year', 'playtime_forever_year': 'Hours Played'})

# Convertir 'Year' a numérico, tratando los valores no numéricos como NaN
combined_user_playtime["Year"] = pd.to_numeric(combined_user_playtime["Year"], errors='coerce')

# Visualizar el DataFrame actualizado
combined_user_playtime

,genres,playtime_forever,user_id,Year,Hours Played
0,Action,1699307,Sp3ctre,0,1657
1,Action,1699307,Sp3ctre,1993,0
2,Action,1699307,Sp3ctre,1995,217
3,Action,1699307,Sp3ctre,1996,0
4,Action,1699307,Sp3ctre,1998,0
...,...,...,...,...,...
165,Utilities,207651,76561198073642113,2014,207651
166,Video Production,168314,ScottyG555,2015,168314
167,Web Publishing,142964,Xyphien,2005,7296
168,Web Publishing,142964,Xyphien,2012,64657


Creamos la función

In [56]:
# Definimos una función auxiliar para normalizar los strings, eliminando tildes y convirtiendo a minúsculas
import unicodedata

def normalize_string(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn').lower()

In [76]:
@app.get('/UserForGenre/{genero}')
def UserForGenre(genero):
    # Normalizar el género de entrada
    genero_norm = normalize_string(genero)

    # Normalizar la columna de géneros en el DataFrame y buscar el usuario con más horas jugadas
    combined_user_playtime['genres_norm'] = combined_user_playtime['genres'].apply(normalize_string)
    top_user = combined_user_playtime[combined_user_playtime['genres_norm'] == genero_norm]["user_id"].iloc[0]

    # Filtrar los datos para el usuario y género seleccionado
    user_history = combined_user_playtime[(combined_user_playtime['user_id'] == top_user) & (combined_user_playtime['genres_norm'] == genero_norm)]

    # Seleccionar solo las columnas necesarias y convertir a diccionario
    user_history_filtered = user_history[['Year', 'Hours Played']].copy()
    user_history_dict = user_history_filtered.to_dict(orient="records")

    # Devolver los resultados
    return {"Usuario": top_user, "con más horas jugadas para": genero, "Historial acumulado": user_history_dict}



In [175]:
UserForGenre('indie')

{'Usuario': 'REBAS_AS_F-T',
 'con más horas jugadas para': 'indie',
 'Historial acumulado': [{'Year': 1999, 'Hours Played': 0},
  {'Year': 2001, 'Hours Played': 11},
  {'Year': 2003, 'Hours Played': 1863},
  {'Year': 2005, 'Hours Played': 0},
  {'Year': 2006, 'Hours Played': 1673},
  {'Year': 2007, 'Hours Played': 1070},
  {'Year': 2008, 'Hours Played': 1366},
  {'Year': 2009, 'Hours Played': 28993},
  {'Year': 2010, 'Hours Played': 21487},
  {'Year': 2011, 'Hours Played': 100155},
  {'Year': 2012, 'Hours Played': 148459},
  {'Year': 2013, 'Hours Played': 169349},
  {'Year': 2014, 'Hours Played': 326927},
  {'Year': 2015, 'Hours Played': 751765},
  {'Year': 2016, 'Hours Played': 814373},
  {'Year': 2017, 'Hours Played': 33887},
  {'Year': None, 'Hours Played': 0},
  {'Year': None, 'Hours Played': 1616}]}

### 3. Función **defUsersRecommend(año:int)**:


In [81]:
@app.get('/UsersRecommend')
def UsersRecommend(year: int):
    """
    Returns the top 3 most recommended games by users for a given year.
    Only considers recommendations where 'reviews.recommend' is True and the sentiment analysis is positive or neutral.

    Parameters:
    year (int): The year to filter the user reviews.

    Returns:
    list: A list of the top 3 most recommended games for the given year.
    """

    # Filter reviews for the given year, recommended, and with positive or neutral sentiment
    filtered_reviews = user_reviews[
        (user_reviews['year'] == year) & 
        (user_reviews['reviews_recommend'] == True) & 
        (user_reviews['sentiment_analysis'].isin([1, 2]))  # 1 for positive, 2 for neutral
    ]

    # Count recommendations by game ID
    recommend_count = filtered_reviews.groupby('reviews_item_id').size().reset_index(name='recommend_count')

    # Sort and get top 3 games
    top_3_games = recommend_count.sort_values(by='recommend_count', ascending=False).head(3)

    # Merge with games dataset to get names
    top_3_games_with_names = pd.merge(top_3_games, steam_games.drop_duplicates(subset='item_id'), 
                                      left_on='reviews_item_id', right_on='item_id')

    # Prepare final result
    top_3_games_list = top_3_games_with_names[['app_name', 'recommend_count']].to_dict(orient='records')
    top_3_result = [{"Puesto 1": top_3_games_list[0]}, 
                    {"Puesto 2": top_3_games_list[1]}, 
                    {"Puesto 3": top_3_games_list[2]}]

    return f"Los 3 juegos más recomendados por usuarios para el año {year} son: {top_3_result}"


In [82]:
UsersRecommend(2014)

"Los 3 juegos más recomendados por usuarios para el año 2014 son: [{'Puesto 1': {'app_name': 'Team Fortress 2', 'recommend_count': 1140}}, {'Puesto 2': {'app_name': 'Counter-Strike: Global Offensive', 'recommend_count': 792}}, {'Puesto 3': {'app_name': 'Dota 2', 'recommend_count': 507}}]"

### 4. Funcion **def UsersWorstDeveloper( año : int )**

Primero, filtramos las reseñas de usuarios para encontrar aquellas que no son recomendadas o tienen sentimientos negativos. Luego, fusiona estos datos con la información de los juegos para asociar las reseñas con los desarrolladores.

In [72]:
@app.get('/UsersWorstDeveloper')
def UsersWorstDeveloper(year: int):
    """
    Función para encontrar los tres desarrolladores de juegos con más reseñas negativas para un año dado.
    Esta función se basa en DataFrames pre-cargados: 'reviews_df' y 'games_df'.

    Parámetros:
    year (int): El año para el cual se buscan los desarrolladores.

    Devuelve:
    list: Los tres principales desarrolladores con más reseñas negativas para el año dado.
    """
    # Filtrar las reseñas para el año dado y donde la recomendación es False
    reviews_year = user_reviews[(user_reviews['year'] == year) & (user_reviews['reviews_recommend'] == False)]

    # Combinar las reseñas con los datos de los juegos para obtener la información del desarrollador
    merged_data = reviews_year.merge(steam_games, left_on='reviews_item_id', right_on='item_id')

    # Contar el número de reseñas negativas para cada desarrollador para el año dado
    developer_negative_reviews = merged_data['developer'].value_counts()

    # Obtener los tres principales desarrolladores con más reseñas negativas
    top_3_developers = developer_negative_reviews.head(3)

    # Formatear la salida
    formatted_output = [{"Puesto 1": top_3_developers.index[0]}, 
                        {"Puesto 2": top_3_developers.index[1]}, 
                        {"Puesto 3": top_3_developers.index[2]}]
    # Construyendo el mensaje
    message = f"Las 3 desarrolladoras con juegos MENOS recomendados por usuarios para el año {year}:"

    return message, formatted_output

In [73]:
UsersWorstDeveloper(2014)

('Las 3 desarrolladoras con juegos MENOS recomendados por usuarios para el año 2014:',
 [{'Puesto 1': 'Bohemia Interactive'},
  {'Puesto 2': 'Facepunch Studios'},
  {'Puesto 3': 'Smartly Dressed Games'}])

### 5. Función def sentiment_analysis (empresa desarrolladora : str)

Para crear la función sentiment_analysis, seguiré estos pasos:

+ **Normalización:** Convertiré tanto el nombre de la desarrolladora proporcionado por el usuario como los nombres de las desarrolladoras en el conjunto de datos a minúsculas para garantizar la comparación sin diferencias de mayúsculas y minúsculas.

+ **Tolerancia a Errores de Escritura:** Para manejar errores menores en la escritura del nombre de la desarrolladora (como errores de ortografía), utilizaré una técnica de comparación de cadenas que permita cierto grado de variación. Una opción es utilizar la distancia de Levenshtein, que mide cuántas ediciones se necesitan para cambiar una cadena en otra.

+ **Búsqueda y Análisis:** La función buscará coincidencias en los nombres de las desarrolladoras, y luego realizará el análisis de sentimiento en las reseñas asociadas, agrupándolas en categorías de 'Negativo', 'Neutral' y 'Positivo'.

+ **Estructura de Resultados:** Finalmente, la función devolverá los resultados en el formato especificado.

In [74]:
@app.get('/sentiment_analysis')
def sentiment_analysis(developer_name: str) -> str:
    """
    Analiza el sentimiento de las reseñas para juegos de una empresa desarrolladora específica.
    
    Args:
    developer_name (str): El nombre de la empresa desarrolladora. La función es insensible a mayúsculas/minúsculas.

    Returns:
    str: Un mensaje con el conteo de reseñas negativas, neutrales y positivas.
    """

    # Normalizar el nombre de la desarrolladora para hacerlo insensible a mayúsculas/minúsculas
    developer_name_normalized = developer_name.lower()

    # Filtrando juegos por la desarrolladora especificada
    filtered_games = steam_games[steam_games['developer'].str.lower() == developer_name_normalized]

    # Obteniendo los IDs de los juegos de la desarrolladora
    game_ids = filtered_games['item_id'].unique()

    # Filtrando las reseñas para los juegos de la desarrolladora
    reviews = user_reviews[user_reviews['reviews_item_id'].isin(game_ids)]

    # Contando las reseñas por análisis de sentimiento
    sentiment_counts = reviews['sentiment_analysis'].value_counts().sort_index()
    sentiment_counts = { "Negative": sentiment_counts.get(0, 0), 
                         "Neutral": sentiment_counts.get(1, 0), 
                         "Positive": sentiment_counts.get(2, 0) }

    # Formateando el mensaje de salida
    return f"El análisis de sentimiento de valor para la desarrolladora {developer_name} es: [Negative = {sentiment_counts['Negative']}, Neutral = {sentiment_counts['Neutral']}, Positive = {sentiment_counts['Positive']}]"

# Ejemplo de cómo usar la función
# resultado = sentiment_analysis("Bohemia Interactive")
# print(resultado)

In [75]:
sentiment_analysis("Bohemia Interactive")

'El análisis de sentimiento de valor para la desarrolladora Bohemia Interactive es: [Negative = 158, Neutral = 223, Positive = 441]'

Prueba

In [24]:
UsersRecommend(2014)

"Los 3 juegos más recomendados por usuarios para el año 2014 son: [{'Puesto 1': {'app_name': 'Team Fortress 2', 'recommend_count': 1140}}, {'Puesto 2': {'app_name': 'Counter-Strike: Global Offensive', 'recommend_count': 792}}, {'Puesto 3': {'app_name': 'Dota 2', 'recommend_count': 507}}]"

In [ ]:
#Prueba

In [13]:
@app.get('/PlayTimeGenre/{genero}')
def PlayTimeGenre(genero):
    # Preparar datos
    genre = steam_games[["item_id", "genres", "year_of_release"]]
    play_time = user_items[["item_id", "playtime_forever"]]

    # Convertir la columna 'item_id' del DataFrame 'play_time' a tipo 'float64'
    play_time['item_id'] = play_time['item_id'].astype('float64')

    # Unificar tablas
    genre_playtime = play_time.merge(genre, on="item_id")

    # Agrupar por género y año, sumar horas jugadas
    total_playtime_by_genre_year = genre_playtime.groupby(["genres", "year_of_release"])["playtime_forever"].sum().reset_index(name='total_playtime')

    # Ordenar y encontrar máximas horas jugadas por género
    sorted_playtime = total_playtime_by_genre_year.sort_values(by="total_playtime", ascending=False)
    max_hours_per_genre = sorted_playtime.groupby("genres")["total_playtime"].agg(MaxHours="max").reset_index()

    # Renombrar columna para el merge y realizar merge
    genre_playtime = sorted_playtime.rename(columns={'total_playtime': 'MaxHours'})
    merged_genre_data = pd.merge(max_hours_per_genre, genre_playtime, on=["genres", "MaxHours"])

    # Convertir a minúsculas para la comparación
    genero_min = genero.lower()
    merged_genre_data["genres"] = merged_genre_data["genres"].str.lower()

    # Filtrar por género específico
    genre_data = merged_genre_data[merged_genre_data["genres"] == genero_min]

    # Encontrar el año con más horas jugadas
    if not genre_data.empty:
        max_playtime_row = genre_data.iloc[0]
        return {"Año de lanzamiento con más horas jugadas para Género " + genero: int(max_playtime_row["year_of_release"])}
    else:
        return {"Error": "Género no encontrado o sin datos"}


In [14]:
PlayTimeGenre('indie')

C:\Users\USER\AppData\Local\Temp\ipykernel_15940\1919246704.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  play_time['item_id'] = play_time['item_id'].astype('float64')


{'Año de lanzamiento con más horas jugadas para Género indie': 2006}

In [15]:
@app.get('/UserForGenre/{genero}')
def UserForGenre(genero):
    # Preparar datos
    user_items['user_id'] = user_items['user_id'].astype('string')
    play_time_user = user_items[["item_id", "playtime_forever", "user_id"]]
    combined_genre_user = play_time_user.merge(genre, on="item_id")

    # Convertir a string y agrupar
    combined_genre_user = combined_genre_user.astype({'genres': 'string', 'year_of_release': 'string', 'user_id': 'string'})
    total_playtime_by_user_year = combined_genre_user.groupby(["genres", "user_id", "year_of_release"])["playtime_forever"].sum().reset_index()
    total_hours_by_user = combined_genre_user.groupby(["genres", "user_id"])["playtime_forever"].sum().reset_index()
    max_hours_by_genre = total_hours_by_user.groupby("genres")["playtime_forever"].agg(playtime_forever="max").reset_index()

    # Combinar y preparar DataFrame
    detailed_max_hours = pd.merge(max_hours_by_genre, total_hours_by_user, on=["genres", "playtime_forever"])
    combined_user_playtime = pd.merge(detailed_max_hours, total_playtime_by_user_year.rename(columns={"playtime_forever": "playtime_forever_year"}))

    # Normalización y búsqueda de usuario
    genero_norm = normalize_string(genero)
    combined_user_playtime['genres_norm'] = combined_user_playtime['genres'].apply(normalize_string)
    user_genre_data = combined_user_playtime[combined_user_playtime['genres_norm'] == genero_norm]

    if user_genre_data.empty:
        return {"Error": "Género no encontrado o sin datos. Por favor, ingresa un género válido."}

    top_user = user_genre_data["user_id"].iloc[0]
    user_history = user_genre_data[user_genre_data['user_id'] == top_user][['year_of_release', 'playtime_forever_year']].rename(columns={'year_of_release': 'Year', 'playtime_forever_year': 'Hours Played'})
    user_history_dict = user_history.to_dict(orient="records")

    # Devolver los resultados
    return {"Usuario": top_user, "con más horas jugadas para": genero, "Historial acumulado": user_history_dict}


In [101]:
@app.get('/UserForGenre/{genero}')
def UserForGenre(genero):
    # Preparar datos
    user_items['user_id'] = user_items['user_id'].astype('string')
    play_time_user = user_items[["item_id", "playtime_forever", "user_id"]]
    combined_genre_user = play_time_user.merge(genre, on="item_id")

    # Convertir a string y agrupar
    combined_genre_user = combined_genre_user.astype({'genres': 'string', 'year_of_release': 'string', 'user_id': 'string'})
    total_playtime_by_user_year = combined_genre_user.groupby(["genres", "user_id", "year_of_release"])["playtime_forever"].sum().reset_index()
    total_hours_by_user = combined_genre_user.groupby(["genres", "user_id"])["playtime_forever"].sum().reset_index()
    max_hours_by_genre = total_hours_by_user.groupby("genres")["playtime_forever"].agg(playtime_forever="max").reset_index()

    # Combinar y preparar DataFrame
    detailed_max_hours = pd.merge(max_hours_by_genre, total_hours_by_user, on=["genres", "playtime_forever"])
    combined_user_playtime = pd.merge(detailed_max_hours, total_playtime_by_user_year.rename(columns={"playtime_forever": "playtime_forever_year"}))

    # Normalización y búsqueda de usuario
    genero_norm = normalize_string(genero)
    combined_user_playtime['genres_norm'] = combined_user_playtime['genres'].apply(normalize_string)
    user_genre_data = combined_user_playtime[combined_user_playtime['genres_norm'] == genero_norm]

    if user_genre_data.empty:
        return {"Error": "Género no encontrado o sin datos. Por favor, ingresa un género válido."}

    top_user = user_genre_data["user_id"].iloc[0]
    user_history = user_genre_data[user_genre_data['user_id'] == top_user][['year_of_release', 'playtime_forever_year']].rename(columns={'year_of_release': 'Year', 'playtime_forever_year': 'Hours Played'})
    user_history_dict = user_history.to_dict(orient="records")

    # Devolver los resultados
    return {"Usuario": top_user, "con más horas jugadas para": genero, "Historial acumulado": user_history_dict}


In [16]:
UserForGenre('indie')

ValueError: You are trying to merge on object and float64 columns for key 'item_id'. If you wish to proceed you should use pd.concat